In [ ]:
# models/llm/fine_tune.ipynb (ячейка)
!pip install -q -r requirements.txt

import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer

# 1. Модель (4-bit)
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model = prepare_model_for_kbit_training(model)

# 2. LoRA config
lora_config = LoraConfig(
    r=16,                  # rank
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Llama-3
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# 3. Токенизатор + промпт
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def formatting_func(example):
    return f"""### Инструкция:
{example['instruction']}

### Текст:
{example['input']}

### Ответ:
{example['output']}"""

# 4. Датасет
dataset = load_dataset("json", data_files={
    "train": "dataset/train.jsonl",
    "validation": "dataset/val.jsonl"
})
dataset = dataset.map(lambda x: {"text": formatting_func(x)})

# 5. Trainer
training_args = TrainingArguments(
    output_dir="./llama-winline-lora",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=512,
    packing=False
)

trainer.train()

# Сохранение адаптера
model.save_pretrained("./llama-winline-lora")
tokenizer.save_pretrained("./tokenizer")